# **富邦新一代API - Node.js範例程式碼**

## SDK 匯入

In [ ]:
const { FubonSDK, BSAction, TimeInForce, OrderType, PriceType, MarketType } = require('fubon-neo');
const sdk = new FubonSDK();

## 登入

In [ ]:
var accounts = sdk.login("H123oooooo", "oooooooo", "./H123oooooo.pfx", "oooooooo");  // 登入帳號 輸入:帳號、密碼、憑證路徑、憑證密碼
console.log(accounts)

### - 選擇與確認登入帳號

In [ ]:
// 自行選擇使用第幾位帳戶
var target_user = accounts.data[0];
console.log(target_user);

## 1. 交易功能

### - 主動回報

In [ ]:
// 訂閱委託回報
sdk.setOnOrder(function(err, content) 
    { console.log("====下單主動回報====\n", err, content)});

// 訂閱委託改動回報(包含改價改量刪單)
sdk.setOnOrderChanged(function(err, content) 
    { console.log("====改單主動回報====\n", err, content)});

// 訂閱成交回報
sdk.setOnFilled(function(err, content) 
    { console.log("====成交主動回報====\n", err, content)})

// 訂閱其他事件回報
sdk.setOnEvent(function(err, content) 
    { console.log("====其他事件主動回報====\n", err, content)})

### - 建立委託單

In [ ]:
var order = {
  buySell: BSAction.Buy,
  symbol: "2330",  // 股票代號
  price: "578",  // 價格
  quantity: 3000,  // 數量
  marketType: MarketType.Common,  // 盤別
  priceType: PriceType.Limit,  // 價格別
  timeInForce: TimeInForce.ROD, 
  orderType: OrderType.Stock
};

console.log(order)

### - 發送委託單

In [ ]:
var res = sdk.stock.placeOrder(target_user, order);
console.log("成功與否: " + res.isSuccess);
if (res.isSuccess) {
    console.log("\n回報內容: " + JSON.stringify(res.data));
} else {
    console.log("\n錯誤訊息: " + res.message);
}

### - 取得所有今日委託紀錄

In [ ]:
var OrderResults = sdk.stock.getOrderResults(target_user);
if (OrderResults.isSuccess) {
    console.log("委託總數量: " + OrderResults.data.length);
}
for (let i = 0; i < OrderResults.data.length; i++) {
    let j = i + 1;
    console.log("第 " + j + " 筆: " + "\n" );
    console.log(OrderResults.data[i]);
    console.log("\n");
}

### - 修改委託價格

In [ ]:
// 先找出可刪改的委託單
var orderRes = sdk.stock.getOrderResults(target_user);
for (let i = 0; i < orderRes.data.length; i++) {
    if (orderRes.data[i].orderNo == "l0002") { //欲修改之委託單書號
        var target_order = orderRes.data[i]
        console.log("i: " + i + " - 委託單號: " + orderRes.data[i].orderNo);
    }
}

console.log(target_order)

In [ ]:
// 修改下面的 orderRes.data[i] -> 根據以上查詢結果，自行輸入 i 編號
var tmpOrder = sdk.stock.makeModifyPriceObj(target_order, "579");
sdk.stock.modifyPrice(target_user, tmpOrder);

### - 修改委託數量

In [ ]:
// 先找出可刪改的委託單
var orderRes = sdk.stock.getOrderResults(target_user);
for (let i = 0; i < orderRes.data.length; i++) {
    if (orderRes.data[i].status == 10) //委託成功
        console.log("i: " + i + " - 委託單號: " + orderRes.data[i].orderNo);
}

In [ ]:
var target_order = OrderResults.data[0]

// 修改下面的 orderRes.data[i] -> 根據以上查詢結果，自行輸入 i 編號
var tmpOrder = sdk.stock.makeModifyQuantityObj(target_order, 50);
sdk.stock.modifyQuantity(target_user, tmpOrder);

### - 刪除委託單

In [ ]:
// 先找出可刪改的委託單
var orderRes = sdk.stock.getOrderResults(target_user);
for (let i = 0; i < orderRes.data.length; i++) {
    if (orderRes.data[i].status == 10) //委託成功
        console.log("i: " + i + " - 委託單號: " + orderRes.data[i].orderNo);
}

In [ ]:
var target_order = OrderResults.data[0]

// 修改下面的 orderRes.data[i] -> 根據前面查詢結果，自行輸入 i 編號
sdk.stock.cancelOrder(target_user, target_order);

### - 查詢歷史委託

In [ ]:
var orderHistory = sdk.stock.orderHistory(target_user, "20240313", "20240313"); // 起始日期與截止日期，目前可查近兩日資料
if (orderHistory.isSuccess) {
    console.log("委託總數量: " + orderHistory.data.length);
}
for (let i = 0; i < orderHistory.data.length; i++) {
    let j = i + 1;
    console.log("第 " + j + " 筆: " + "\n" );
    console.log(orderHistory.data[i]);
    console.log("\n");
}

### - 查詢歷史成交資訊

In [ ]:
var filledHistory = sdk.stock.filledHistory(target_user, "20240313", "20240313"); // 起始日期與截止日期，目前可查近兩日資料
if (filledHistory.isSuccess) {
    console.log("成交總數量: " + filledHistory.data.length);
}
console.log(filledHistory);

### - 資券配額查詢

In [ ]:
sdk.stock.marginQuota(target_user, "2330");

### - 現冲券配額查詢

In [ ]:
sdk.stock.daytradeAndStockInfo(target_user, "2330");

### - 批次下單

#### -- 建立批次委託單

In [ ]:
var orders = [{
    buySell: BSAction.Buy,
    symbol: "1101",
    price: "36",
    quantity: 2000,
    marketType: MarketType.Common,  // 盤別
    priceType: PriceType.Limit,
    timeInForce: TimeInForce.ROD,
    orderType: OrderType.Stock  
  },{
    buySell: BSAction.Buy,
    symbol: "2330",
    price: "579",
    quantity: 5000,
    marketType: MarketType.Common,
    priceType: PriceType.Limit,
    timeInForce: TimeInForce.ROD,
    orderType: OrderType.Stock
  },
];

#### -- 批次下單並查看回報

In [ ]:
// 批次下單
var res = sdk.stock.batchPlaceOrder(target_user, orders);

if (res.isSuccess) {
    console.log("批次委託送單成功");
    console.log(JSON.stringify(res.data))
} else {
    console.log("批次委託送單失敗: " + res.message);
}

#### -- 取得批次委託列表

In [ ]:
// 查詢批次委託列表
var batch_res = sdk.stock.batchOrderLists(target_user);
if (batch_res.isSuccess) {
    console.log("共有" + batch_res.data.length + "筆批次下單");
    for (let i = 0; i < batch_res.data.length; i++) {
        console.log(i + " - 批次委託序號: " + batch_res.data[i].batchSeqNo);        
    }
} else {
    console.log("查詢委託失敗: " + res.message);
}

#### -- 取得批次委託送單明細

***註：*** 此僅為送單紀錄，無交易狀態更新

In [ ]:
var target_batch_order = batch_res.data[0]

In [ ]:
// 由列表中的項目，來查詢批次委託的細項
res = sdk.stock.batchOrderDetail(target_user, target_batch_order);

if (res.isSuccess) {
    for (let i = 0; i < res.data.length; i++) {
        console.log(i);
        console.log("股票代號: " + res.data[i].stockNo);
        console.log("委託數量: " + res.data[i].quantity);
        if (res.data[i].hasOwnProperty('price')) {
            console.log("委託價格: " + res.data[i].price);
        } else {
            console.log("委託價格: " + res.data[i].priceType);
        }
        if (res.data[i].hasOwnProperty('orderNo')) {
            console.log("委託單號: " + res.data[i].orderNo);
        } else {
            console.log("委託失敗: " + res.data[i].errorMessage);
        }
        
    }
} else {
    console.log("查詢失敗: " + res.message);
}

#### -- 批次改價

In [ ]:
// 批次改價: 用上面批次委託明細來填入下面的批此修改價格
var modify_pris = [ 
    sdk.stock.makeModifyPriceObj(res.data[0], "37"),
    sdk.stock.makeModifyPriceObj(res.data[1], "578"),
];

sdk.stock.batchModifyPrice(target_user, modify_pris);

#### -- 批次改量

In [ ]:
// 批次改量: 用上面批次委託明細來填入下面的批此修改數量
var modify_qtys = [ 
    sdk.stock.makeModifyQuantityObj(res.data[0], 1000),
    sdk.stock.makeModifyQuantityObj(res.data[1], 1000),
];

sdk.stock.batchModifyQuantity(target_user, modify_qtys);

#### -- 批次刪除委託單

In [ ]:
// 把所有委託成功且未成交的單子找出來
function get_succ_order(target_user) {
    var orderRes = sdk.stock.getOrderResults(target_user);
    var olist = [];
    for (let i = 0; i < orderRes.data.length; i++) {
        if (orderRes.data[i].status == 10) //委託成功
            olist.push(orderRes.data[i]);
    }
    return olist;
}

var olist = get_succ_order(target_user);
// 批次刪單
sdk.stock.batchCancelOrder(target_user, olist);

## 2. 帳務功能

### - 庫存查詢

In [ ]:
var res = sdk.accounting.inventories(target_user);
if (res.isSuccess) {
    console.log("庫存總數量: " + res.data.length);
}
for (let i = 0; i < res.data.length; i++) {
    let j = i + 1;
    console.log("第 " + j + " 筆: " + "\n" );
    console.log(res.data[i]);
    console.log("\n");
}

### - 未實現損益查詢

In [ ]:
sdk.accounting.unrealizedGainsAndLoses(target_user);

### - 已實現損益彙總

In [ ]:
sdk.accounting.realizedGainsAndLosesSummary(target_user);

### - 已實現損益明細

In [ ]:
sdk.accounting.realizedGainsAndLoses(target_user);

### - 維持率查詢

In [ ]:
sdk.accounting.maintenance(target_user);

### - 交割款查詢

In [ ]:
sdk.accounting.querySettlement(target_user,"3d");  // 3d or 0d

### - 銀行餘額查詢

In [ ]:
sdk.accounting.bankRemain(target_user);

## 3. 行情功能

### 建立行情連線

In [ ]:
sdk.initRealtime();

In [ ]:
var client = sdk.marketdata.restClient;

### Web API 日內行情

### - Intraday Tickers 股票、指數列表

In [ ]:
client.stock.intraday
  .tickers({ type: "EQUITY", exchange: "TWSE", isNormal: true })
  .then((data) => console.log(data));

### - Intraday Ticker 股票資訊

In [ ]:
// 可根據以上查到的股票代號填入
client.stock.intraday.ticker({ symbol: '2330' })
  .then(data => console.log(data));

### - Intraday Quote 即時報價

In [ ]:
client.stock.intraday.quote({ symbol: '2330' })
  .then(data => console.log(data));

### - Intraday Candles 分K資料

In [ ]:
client.stock.intraday.candles({ symbol: '2330', timeframe: "5" })  // time frame 可選 1 5 10 15 30 60
  .then(data => console.log(data));

### - Intraday Trades 交易明細

In [ ]:
client.stock.intraday.trades({ symbol: '2330', limit: 10 })  // limit: 取得最新的n筆資料，預設為50
  .then(data => console.log(data));

### - Intraday Volumes 分價量表

In [ ]:
client.stock.intraday.volumes({ symbol: '2330' })
  .then(data => console.log(data));

### Web API 行情況照

### - Snapshot Quotes 市場行情快照

In [ ]:
client.stock.snapshot.quotes({ market: 'TSE' })
  .then(data => console.log(data));

### - Snapshot Movers 漲跌幅排行

In [ ]:
client.stock.snapshot.movers({ market: 'OTC', direction: 'up', change: 'percent' })
  .then(data => console.log(data));

### - Snapshot Actives 成交量值排行

In [ ]:
client.stock.snapshot.actives({ market: 'OTC', trade: 'volume' })
  .then(data => console.log(data));

### Web API 歷史行情

### - Historical Candles 歷史K線資料

In [ ]:
client.stock.historical.candles(
    { symbol: '2330', from: '2023-07-26', to: '2024-01-30', fields: 'open,close,volume', timeframe: "D" })
    .then(data => console.log(data));

### - Historical Stats 近 52 週股價數據

In [ ]:
client.stock.historical.stats({ symbol: '2330' })
  .then(data => console.log(data));

### Web Socket 

### 訂閱接收最新資訊

In [ ]:
// 處理各種連接狀況
function handle_connect() {
    console.log("Web Socket 連接成功");
}
function handle_disconnect(){
    console.log("Web Socket 連接斷線");
}
function handle_error(error) {
    console.log("Web Socket 錯誤: " + error);
}
var subscribe_ids = [];
function handle_message(text){
    try {
        let msg = JSON.parse(text);
        let event = msg.event;
        let data = msg.data;
        
        if (event == "data") {
            console.log("[" + msg.channel + "] " + JSON.stringify(data));
        } else {
            if (event == "subscribed") {
                let id = data.id;
                if (subscribe_ids.includes(id))
                    console.log("[Error] 訂閱 ID " + id + " 重複訂閱");
                else {
                    subscribe_ids.push(id);
                }
            } else if (event == "unsubscribed") {
                let id = data.id;
                const loc = subscribe_ids.indexOf(id);
                if (loc >= 0) {
                    subscribe_ids.splice(loc, 1);
                } else {
                    console.log("[Error] 查無此筆訂閱 ID: " + id);
                }
            }
            console.log(text);
        }
    }
    catch (err) {
        console.log("[-] " + text);
        handle_error("parsing JSON: " + err);
    }
}

### 開始訂閱

In [ ]:
const stock = sdk.marketdata.webSocketClient.stock;
stock.on("connect", handle_connect);
stock.on("message", handle_message);
stock.on("disconnect", handle_disconnect);
stock.on("error", handle_error);

stock.connect();  // WebSocket 連線開始

In [ ]:
// 訂閱交易明細
stock.subscribe({
    'channel': 'trades',
    'symbol': '5439'
});

In [ ]:
// 訂閱最佳五檔報價
stock.subscribe({ 
    'channel': 'books', 
    'symbol': '5439'
});

In [ ]:
// 訂閱股票聚合數據
stock.subscribe({ 
    'channel': 'indices', 
    'symbol': 'IR0001'
});

### 取消訂閱

In [ ]:
// 可由此看到目前已經訂閱的 channel ID
console.log(subscribe_ids);

In [ ]:
// 上面拿到的 ID 填進來即可取消訂閱
stock.unsubscribe({"id": "JwqPM29V2VUZKWN8KWJvc2PGm3qYzEFj9B6ygMNwH5k"});

In [ ]:
// 取消掉所有的訂閱
var tmp = Array.from(subscribe_ids);
for (let i = 0; i < tmp.length; i++) {
    stock.unsubscribe({"id": tmp[i]});
}

In [ ]:
// 斷線
stock.disconnect();